In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from urllib.parse import urlencode
import datetime

# Load environment variables in a file called .env

load_dotenv(override=True)

api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please get your api key and reference it in .env!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [2]:
openai = OpenAI()

In [7]:
# Sending first message to the OpenAI

message = "Hello, OpenAI GPT! This is my first time connecting to you!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role": "user", "content": message}])

print(response.choices[0].message.content)

Hello! Welcome! I'm glad you're here. How can I assist you today?


In [12]:
# asking some math questions

messages = [
    {"role": "system", "content": "you are a math expert who should help with math questions, with sarcasm"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [13]:
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, wow, what a challenge! Are you ready for this? Drumroll, please... It's 4! Shocking, I know. If only all math was this mind-blowing!


In [3]:
# A Function to scrape a website

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [4]:
result = Website("https://www.reddit.com/")
print(result.title)

Reddit - The heart of the internet


In [6]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n"
    user_prompt += website.text
    return user_prompt

In [7]:
def messages_for(website): 
    prompt = user_prompt_for(website)
    if not prompt:
        raise ValueError("user_prompt_for(website) returned an empty string!")
    
    return [
        {"role": "system", "content": "You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown."},
        {"role": "user", "content": prompt}
    ]

In [8]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [10]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [11]:
display_summary("https://reddit.com")

# Reddit - The Heart of the Internet

Reddit is a social media platform that serves as a community hub for discussions on a vast array of topics. Users can participate in forums, share news, and engage with others around shared interests.

## Recent News Highlights

- **US Tariffs on Solar Panels**: The US imposes tariffs as high as 3,521% on solar imports from Southeast Asia.
  
- **Politics**: Harvard University has filed a lawsuit against the Trump administration.

- **Academy Awards Update**: A new rule announced by the Oscars states that voters must see all nominated films to cast their vote.

- **Sports Update**: The Detroit Pistons recently defeated the New York Knicks in an NBA matchup, accompanied by discussions around a controversial play involving Jalen Brunson.

- **WWE Update**: Photos from Wrestlemania Weekend have been shared.

- **Technology**: The FTC has sued Uber for charging users for Uber One without their consent.

## Community Highlights

Reddit also features various community-driven discussions, including personal conflicts in threads like "AITA for refusing to apologize for slapping my boyfriend." This thread narrates a birthday party incident that escalated into a personal conflict, highlighting issues of respect and relationships.

Overall, Reddit acts as a platform for users to stay informed, share experiences, and connect over a wide range of topics.

# Stock Analysis using Yahoo Finance

In [69]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class YahooFinanceWebsite:
    def __init__(self, stock_symbol):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.stock_symbol = stock_symbol.upper()

    def __build_url(self, params):
        base_url = f"https://finance.yahoo.com/quote/{self.stock_symbol}/history/"
        query_string = urlencode(params)
        return f"{base_url}?{query_string}"

    def get_stock_data(self):
        datetime_now = datetime.datetime.now()
        datetime_year_ago = datetime_now - datetime.timedelta(days=365)
        params = {"frequency": "1wk", "period1": datetime_year_ago.timestamp(), "period2": datetime_now.timestamp()}
        url = self.__build_url(params)
        response = requests.get(url, headers=headers)

        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        html_table_data = soup.find("table")

        return title, html_table_data

In [59]:
def build_stock_analysis_prompt(stock_symbol, title, stock_table_data):
    sys_prompt = r"""You are an assistant that analyzes the contents of HTML formated table that contains data on a specific stock.
    The HTML table contains the date, open price, close price, low and highs aggregated for every week over one year timeframe.
    Ignoring text, tags or html attributes that might be navigation related. 
    Respond in Markdown format"""

    user_prompt = f"The data provided below in the HTML table format for {stock_symbol}: {title} from the Yahoo Finances.\
    Make the explaination easy enough for a newbie to understand. \
    Analyze and Summarize the trends on this stock:\n{stock_table_data}\n\n\
    Also, calculate the total returns in percentage one could have expected over this period."

    return [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [62]:
def analyze_stock_trends(stock_symbol):
    stock_data_page = YahooFinanceWebsite(stock_symbol)
    title, stock_table_data = stock_data_page.get_stock_data()
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = build_stock_analysis_prompt(stock_symbol, title, stock_table_data)
    )
    return response.choices[0].message.content

def display_analysis(stock_symbol):
    display(Markdown(analyze_stock_trends(stock_symbol)))

In [70]:
display_analysis("GOOG")

# Alphabet Inc. (GOOG) Stock Analysis

Here’s a summary and analysis of Alphabet Inc. (GOOG) based on the provided stock prices over a year.

## Overview of Data

The data includes weekly stock prices, and for each week we have:
- **Opening Prices:** The price at which the stock opened at the beginning of the week.
- **Closing Prices:** The price at which the stock closed at the end of the week.
- **High Prices:** The highest price the stock reached during the week.
- **Low Prices:** The lowest price the stock reached during the week.

## Trends Observed

1. **Price Fluctuation:**
   - Throughout the year, the stock prices have shown significant fluctuation, with noticeable increases and decreases week over week.
   - For instance, the stock's price opened at **202.21** on Feb 3, 2025, indicating a higher level compared to weeks earlier in the data. The highest recorded price during the week was **208.70**.

2. **Price Movement:**
   - The stock price has generally trended upwards in the early parts of the year, reaching peaks of around **208.70** in early February.
   - There were notable dips, for example, in late December 2024, where the price was reported as low as **170.49**.

3. **Volume Changes:**
   - The trading volume has varied significantly across the weeks, with some weeks reporting volumes in excess of **139 million** shares traded, reflecting either increased investor interest or volatility around certain events or announcements.

4. **Dividends:**
   - There were dividends paid out during this period, notably **0.20** on multiple occasions. This is an additional return to investors aside from the capital gains from price appreciation.

## Total Returns Calculation

To compute the total returns percentage over the time period:
1. **Initial Price**:  The price from Feb 12, 2024 (the first recorded week in the data) where the stock closed at **141.76**.
2. **Final Price**: The price from Feb 3, 2025, closing at **187.14**.
3. **Dividend Returns**:  Calculate total dividends received:  **0.20 + 0.20 + 0.20 = 0.60** (paid three times).

### Total Returns Formula

\[
\text{Total Returns Percentage} = \left(\frac{\text{Final Price} + \text{Total Dividends} - \text{Initial Price}}{\text{Initial Price}}\right) \times 100
\]

### Substituting Numbers

\[
\text{Total Returns Percentage} = \left(\frac{187.14 + 0.60 - 141.76}{141.76}\right) \times 100 \approx \left(\frac{46.98}{141.76}\right) \times 100 \approx 33.13\%
\]

## Conclusion

The analysis indicates that Alphabet Inc. (GOOG) has performed well over the observed period, yielding a favorable total return of approximately **33.13%**. The stock has shown substantial volatility, with highs and lows, making it a potentially exciting option for investors who can tolerate risk. The dividends add to the returns, making it attractive even beyond just price appreciation. 

Overall, despite the fluctuations and various macroeconomic factors that may influence stock prices, Alphabet Inc. remains a strong contender in the market.

In [64]:
display_analysis("CRM")

## Analysis of Salesforce, Inc. (CRM) Stock Trends

### Overview

The table presents the historical stock prices for Salesforce, Inc. (CRM) for various weeks, including key data points: **Open**, **High**, **Low**, **Close**, **Adjusted Close**, and **Volume**. 

### Key Observations

- **Recent Prices**: 
  - In February 2025, the stock opened at **$332.00** and closed at **$325.83**. 
  - There were fluctuations in prices with highs reaching **$348.04** and lows dropping to **$323.72** during the early February week.

- **Trends Over Time**:
  - The stock experienced significant variation over the year. 
  - A noticeable peak was observed in **December 2024** where the stock reached a high of **$358.07** and closed at **$343.65**, indicating some upward momentum during this period. 
  - The lowest recent price was **$212.00** during **late May 2024**, suggesting a notable drop from earlier highs, with a close price of **$234.44**.

### Price Movements

- **Overall Movement**: 
  - The stock prices have shown a general trend of rising over the long term but with significant drops and rises evident.
  - Frequent fluctuations in the high and low prices indicate demand volatility.
  
- **Volume Analysis**: 
  - The trading volume varied greatly, with the highest volume of **126,360,300** recorded on **May 27, 2024**. High trading volumes often accompany volatility and significant news or events affecting the stock.

### Return Calculation

To calculate the **total returns** over the time period of the data (from the lowest point in May 2024 to the most recent recorded price in February 2025):

1. **Lowest Closing Price**:
   - **May 27, 2024**: Closed at **$234.44**.

2. **Recent Closing Price**:
   - **Feb 7, 2025**: Closed at **$325.83**.

3. **Return Calculation**: 
   \[
   \text{Total Return} = \left( \frac{\text{Final Price} - \text{Initial Price}}{\text{Initial Price}} \right) \times 100
   \]
   \[
   \text{Total Return} = \left( \frac{325.83 - 234.44}{234.44} \right) \times 100 \approx 38.95\%
   \]

### Summary of Trends

- The stock has undergone significant ups and downs throughout the year.
- Despite dips, including a low of **$234.44**, the stock has made a considerable recovery, reaching around **$325.83**, leading to a **total return of approximately 38.95%**.
- Investors looking at CRM's performance should pay attention to periodic price fluctuations and the influences of earnings reports, market conditions, and broader economic factors.

### Conclusion

Salesforce's stock reflects both volatility and recovery potential, appealing to both short-term traders looking for value and long-term investors. A closer look at future earnings announcements and market conditions will be crucial for predicting the next direction of CRM shares.

# Create Social Media Posts based on event brite meetups

In [73]:
# Create Social Media Posts based on event brite meetups

url = "https://www.eventbrite.ie/e/startups-investors-networking-san-francisco-tickets-801798570107?aff=ebdssbdestsearch&keep_tld=1"

def get_prompts(summary):
    system_prompt = r"""You are an assitant who analyzes the contents of an event and provide short summaries to post about the events on Social Media websites like Twitter, Instagram and Facebook. \
        Ensure the summaries respect the platform rules."""
    
    user_prompt = f"The events details are as follows: {summary}. Please summarize and capture details like time and location."

    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

summary = summarize(url)

response = openai.chat.completions.create(model="gpt-4o-mini", messages=get_prompts(summary))

display(Markdown(response.choices[0].message.content))

### Twitter
🚀 Exciting networking opportunity for startups and investors! Join us at the "Startups & Investors Networking" event on Feb 19, 6-9 PM PST at The Devil's Acre, SF. Pitch your ideas, connect with top investors, and gain valuable insights! 🍹 #StartupValley #Networking

---

### Instagram
🌟 Ready to elevate your startup? Join us for the **Startups & Investors Networking** event hosted by **Startup Valley** on **February 19** from **6 PM to 9 PM PST** at **The Devil's Acre** in San Francisco! Mingle with investors, pitch your innovative projects, and expand your network. Don't miss your chance to shine! ✨ Tickets are limited, so secure yours now! 🍹 #StartupValley #InvestInYourFuture #NetworkingEvent

---

### Facebook
📢 Calling all startups! Join us for the **"Startups & Investors Networking"** event on **February 19** from **6:00 PM to 9:00 PM PST** at **The Devil's Acre** in San Francisco. This is your chance to connect with investors, share your pitch, and gain valuable insights from industry professionals. Enjoy a welcoming atmosphere and the opportunity to be recognized as the "Most Investment-Worthy" startup! Don't forget, you must be **21 or older** and purchase a drink at the venue. Get your tickets now and secure your spot! 🥂 #StartupValley #NetworkingEvents

# Youtube Video Transcript Summary

In [77]:
# Youtube Video Transcript Summary
from youtube_transcript_api import YouTubeTranscriptApi
import re

class YoutubeVideoID:
    def __init__(self, url):
        self.url = url
        self.video_id = self.extract_video_id(url)

    def extract_video_id(self, url):
        """
        Extracts the YouTube video ID from a given URL.
        Supports both regular and shortened URLs.
        """
        # Regular expression to match YouTube video URL and extract the video ID
        regex = r"(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|\S*\?v=)|(?:youtu\.be\/))([a-zA-Z0-9_-]{11})"
        match = re.match(regex, url)
        
        if match:
            return match.group(1)
        else:
            raise ValueError("Invalid YouTube URL")

    def __str__(self):
        return f"Video ID: {self.video_id}"


In [79]:
video_url = "https://www.youtube.com/watch?v=KrRD7r7y7NY"

yt_video = YoutubeVideoID(video_url)
print(yt_video)

Video ID: KrRD7r7y7NY


In [80]:
def get_transcript(video_id, language='en'):
    try:
        # Try to get the transcript in the desired language (Indonesian by default)
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[language])
        # Join all the 'text' fields into a single string
        return " ".join([item['text'] for item in transcript])
    except Exception as e:
        print(f"Error fetching transcript: {e}")
        return None

In [81]:
# Fetch transcript using the video ID
transcript_text = get_transcript(yt_video.video_id)
print(len(transcript_text))

24651


In [82]:
# Function to summarize text using ChatGPT
def summarize_text(text):
    try:
        system_prompts = """
        You are a helpful assistant who provides concise and accurate summaries of text. Your task is to:
        
        - Capture the key points of the content.
        - Keep the summary brief and easy to understand with proper insights
        - Use bullet points where appropriate to enhance clarity and structure.
        """
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompts},
                {"role": "user", "content": f"Summarize the following text:\n{text}"}
            ],
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error summarizing text: {e}")
        return None

In [83]:
summary = summarize_text(transcript_text)

display(Markdown(summary))

**Summary of Andrew's Presentation at Snowflake Build:**

- **AI as a Fundamental Technology:**
  - AI is likened to electricity, a transformative general-purpose technology with vast applications.
  - The focus on foundational technologies is critical, but application development is essential for maximizing value.

- **AI Opportunity Landscape:**
  - The AI stack consists of semiconductors, cloud infrastructure, foundation models, and applications.
  - Emphasis on the application layer where significant opportunities to generate revenue lie.

- **Rapid Development with Generative AI:**
  - Generative AI allows for faster machine learning model development.
  - Traditional workflows taking months can now be reduced to days for prototyping and deployment.

- **Fast Experimentation:**
  - Encourages quick prototyping and testing multiple ideas, leading to innovative solutions.
  - New workflows collect and utilize data in parallel to accelerate development.

- **Agentic AI Workflows:**
  - Introduction of agentic AI as a key trend.
  - Agentic workflows involve iterative processes that improve output by addressing tasks in multiple stages (e.g., revising essays, processing complex documents).

- **Four Major Design Patterns for Agentic Workflows:**
  1. **Reflection** - Continuous feedback and improvement of generated outputs.
  2. **Tool Use** - Enabling models to request and execute external functions or tools.
  3. **Planning** - Structuring complex tasks into actionable steps.
  4. **Multi-agent Collaboration** - Utilizing multiple AI agents to address different aspects of a task together.

- **Visual AI Advances:**
  - Demonstrated the capability to process visual data and generate actionable insights using agentic workflows.
  - The importance of visual data processing is rising, with tools available to automate and derive value from image and video data.

- **Future AI Trends:**
  -  Emphasis on efficient token generation and enhancements to large language models for versatile application.
  - Growing significance of data engineering to manage unstructured data alongside burgeoning visual AI capabilities.

- **Conclusion:**
  - Current advancements allow builders to innovate and create applications that were previously unattainable.
  - Invitation to explore visual AI demos on the provided platform for hands-on experience. 

Andrew’s enthusiasm reinforces that this is a pivotal moment for AI development and application building.

# Tool to answer any technical questions related to software engineering using open AI

In [85]:
question = """Please explain how do you solve the prime numbers in Java efficiently"""

messages = [
    {"role": "system", "content": "you are a programming expert who answers questions about python, java, software engineering, LLMs and Data science"},
    {"role": "user", "content": "Please give details explanations of the followig question:" + question}
]


In [86]:
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

To find prime numbers efficiently in Java, you can use several methods, each improving on the straightforward techniques. A prime number is defined as a number greater than 1 that has no positive divisors other than 1 and itself.

### 1. **Understanding Prime Numbers**:
   - A prime number is a natural number greater than 1 that cannot be formed by multiplying two smaller natural numbers. Examples include 2, 3, 5, 7, etc.

### 2. **Basic Methodology**:
   - The simplest way to check if a number \( n \) is prime is to check if it is divisible by any number from 2 to \( n-1 \). However, this is inefficient for larger numbers.

### 3. **Optimized Solution: Trial Division**:
   - **Check up to the Square Root**: You only need to check divisibility up to the square root of \( n \). If \( n \) is divisible by any number up to \( \sqrt{n} \), it cannot be prime.

### 4. **Skipping Even Numbers**:
   - After checking for divisibility by 2, you can skip all even numbers and check only the odd n

In [89]:
# Get gpt-4o-mini to answer, with streaming

stream = openai.chat.completions.create(model='gpt-4o-mini', messages=messages,stream=True)
    
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

Finding prime numbers efficiently is a common problem in computer science and programming. When it comes to Java, there are multiple algorithms you can implement to generate and check for prime numbers efficiently. Below, I'll explain some popular methods, along with code examples to help you implement them.

### Basic Concepts of Prime Numbers

A prime number is a natural number greater than 1 that cannot be formed by multiplying two smaller natural numbers. The only factors of a prime number are 1 and itself.

### Efficient Methods for Finding Prime Numbers

1. **Trial Division**:
   This is the simplest method to check if a number is prime. You can check for factors from 2 to the square root of the number.

   **Algorithm**:
   - If the number \( n \) is less than 2, it's not prime.
   - Check divisibility from 2 to \( \sqrt{n} \).
   - If you find a divisor, it's not prime; otherwise, it is.

   **Java Implementation**:
   java
   public class PrimeChecker {
       public static boolean isPrime(int n) {
           if (n <= 1) return false;
           for (int i = 2; i * i <= n; i++) {
               if (n % i == 0) {
                   return false;
               }
           }
           return true;
       }
   }
   

2. **Sieve of Eratosthenes**:
   This is a highly efficient algorithm to find all prime numbers up to a specified integer \( n \). It works by iteratively marking the multiples of each prime number starting from 2.

   **Algorithm**:
   - Create a boolean array `prime[0..n]` and initialize all entries as true. A value in prime[i] will finally be false if \( i \) is Not a prime.
   - Start from the first prime number (2).
   - Mark all its multiples as non-prime.
   - Move to the next number that is still true, and repeat the process.
   - Continue until you have processed numbers up to \( \sqrt{n} \).

   **Java Implementation**:
   java
   import java.util.Arrays;

   public class SieveOfEratosthenes {
       public static void findPrimes(int n) {
           boolean[] prime = new boolean[n + 1];
           Arrays.fill(prime, true);
           prime[0] = prime[1] = false; // 0 and 1 are not prime numbers

           for (int i = 2; i * i <= n; i++) {
               if (prime[i]) {
                   for (int j = i * i; j <= n; j += i) {
                       prime[j] = false;
                   }
               }
           }

           // Printing all prime numbers
           for (int i = 2; i <= n; i++) {
               if (prime[i]) {
                   System.out.print(i + " ");
               }
           }
       }
   }
   

3. **Optimized Trial Division**:
   You can further optimize the trial division method by only checking odd numbers after checking for 2. Also, you can skip checking even numbers.

   **Java Implementation**:
   java
   public class OptimizedPrimeChecker {
       public static boolean isPrime(int n) {
           if (n <= 1) return false;
           if (n == 2) return true; // 2 is prime
           if (n % 2 == 0) return false; // exclude even numbers

           for (int i = 3; i * i <= n; i += 2) {
               if (n % i == 0) {
                   return false;
               }
           }
           return true;
       }
   }
   

### Performance Considerations

- **Complexity**: The Sieve of Eratosthenes is \( O(n \log \log n) \), which makes it much faster than trial division \( O(\sqrt{n}) \) for larger values of \( n \).
- **Memory Usage**: The Sieve of Eratosthenes requires \( O(n) \) additional memory due to the boolean array it uses to store the prime status of each number.

### Conclusion

When solving for prime numbers in Java, the approach you choose depends on your specific requirements, such as the range of numbers and performance considerations. For generating a large list of primes, the Sieve of Eratosthenes is often the best choice, while the trial division method (especially the optimized version) is simple and effective for checking if individual numbers are prime.